In [ ]:
import pandas as pd
from deepctr.inputs import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

In [ ]:
data = pd.read_csvdata = pd.read_csv("./movielens_sample.txt")
#print(type(data))
sparse_features = ["movie_id", "user_id",
                       "gender", "age", "occupation", "zip", ]
SEQ_LEN = 50
negsample = 0

In [ ]:
# 1. 首先对于数据中的特征进行ID化编码，然后使用 `gen_date_set` and `gen_model_input`来生成带有用户历史行为序列的特征数据
features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1

user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')

item_profile = data[["movie_id"]].drop_duplicates('movie_id')

user_profile.set_index("user_id", inplace=True)

user_item_list = data.groupby("user_id")['movie_id'].apply(list)

train_set, test_set = gen_data_set(data, negsample)

train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)


1、 LabelEncoder:将离散型的数据转换成 0 到 n−1 之间的数，这里 n 是一个列表的不同取值的个数
这里对user_id, movie_id, gender, age, occupation, zip都进行了one-hot映射，并且+1了，所以取值从1开始
其余的字段data数据中保留原来的值

2、feature_max_idx用于存每个特征的最大值+1，
这里特征：'user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip'

3、dataFrame前面的第一列可以看成是行号

4、user_profile是根据user_id去重之后取用户的gender, age, occupation, zip用户通用特征
同样item_profile特征是根据movie_id特征去重之后取物品的id特征

5、user_item_list根据user_id进行groupby取到用户的物品列表

In [ ]:
from tqdm import tqdm
import random
def gen_data_set(data, negsample=0):

    data.sort_values("timestamp", inplace=True) #这里需要根据时间进行排序,从小到大进行排序
    item_ids = data['movie_id'].unique()

    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')): # reviewerID就是key， hist就是iter 
        pos_list = hist['movie_id'].tolist() #每个用户的电影列表 
        rating_list = hist['rating'].tolist() #每个电影的评分列表
        #time_list = hist['timestamp'].tolist()
        #print(hist[::-1]) #从最后一个元素到第一个元素复制一遍，即倒序
        #print(pos_list)
        #print(rating_list)

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list)) #全量的item_ids和浏览过的item_ids去重
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True) #这里是负样本采样
        for i in range(1, len(pos_list)):
            hist = pos_list[:i] #这里的做法相当于扩大了列表的数据
            if i != len(pos_list) - 1: #如果这个电影不是最后一个就加到训练样本, 用户号，X, 电影号，1abel, 浏览个数，当前评分
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]),rating_list[i])) #这里为什么要倒叙？
                #for negi in range(negsample):
                #    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else: #否则的话加到测试样本
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))

    random.shuffle(train_set) #用户，看过的列表，最后一个电影，1，看过列表的长度，当前的评分
    random.shuffle(test_set) 

    return train_set,test_set

6、gen_data_set
6.1	限根据时间戳进行排序
6.2 对于每个用户的浏览序列，去除样本列表和评分列表
6.3 从头遍历扩展样本列表，当不是最后一个就加到训练样本中，否则加到测试样本中
6.4 这里每个样本 uid, 子序列（倒叙，不清楚为什么要倒叙），待预测的样本（电影号），label，序列长度，当前评分
6.5 然后对样本进行随机打散

In [ ]:
import numpy as np
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
def gen_model_input(train_set,user_profile,seq_max_len): 
    train_uid = np.array([line[0] for line in train_set]) #这里提取训练数据每一列的值 
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)  
    # 大于此长度的序列将被截短，小于此长度的序列将在后部填0
    # pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补`
    #‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
    # value：浮点数，此值将在填充时代替默认的填充值0
    
    train_model_input = {"user_id": train_uid, "movie_id": train_iid, "hist_movie_id": train_seq_pad,
                         "hist_len": train_hist_len}

    for key in ["gender", "age", "occupation", "zip"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values
    
    return train_model_input, train_label

7、gen_model_input
7.1 去每个样本每一列的值
7.2 pad_sequences，参数maxlen=50，最大长度为50，padding=post表明在尾部补0，truncating=post表示在尾部进行截断
7.3 用一个字典train_model_input保存用户的user_id, movie_id, padding过的序列hist_movie_id， 序列长度hist_len
7.4 利用python的语法特性主要是loc，把用户的特征拼接到字典train_model_input中

In [ ]:
# 2. 配置一下模型定义需要的特征列，主要是特征名和embedding词表的大小

embedding_dim = 16

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
                        SparseFeat("gender", feature_max_idx['gender'], embedding_dim),
                        SparseFeat("age", feature_max_idx['age'], embedding_dim),
                        SparseFeat("occupation", feature_max_idx['occupation'], embedding_dim),
                        SparseFeat("zip", feature_max_idx['zip'], embedding_dim),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]


8、用sparseFeat或者VarLenSparseFeat来定义模型的特征，这里只是定义，没有传数据，主要定义了名字、特征最大下标，embedding的维度
user_feature_columns：user_id，gender, age, occupation,zip, hist_id
item_feature_columns: movie_id

In [ ]:
# 3. 定义一个YoutubeDNN模型，分别传入用户侧特征列表`user_feature_columns`和物品侧特征列表`item_feature_columns`。然后配置优化器和损失函数，开始进行训练。

K.set_learning_phase(True)

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=5, user_dnn_hidden_units=(64, 16))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=True,p=1,k_max=2,num_sampled=5,user_dnn_hidden_units=(64,16),init_std=0.001)

model.compile(optimizer="adagrad", loss=sampledsoftmaxloss)  # "binary_crossentropy")

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=256, epochs=1, verbose=1, validation_split=0.0, )
#print(history)


9、定义了YoutubeDNN的网络，在fit的时候实际传入参数


In [ ]:
# 4. 训练完整后，由于在实际使用时，我们需要根据当前的用户特征实时产生用户侧向量，并对物品侧向量构建索引进行近似最近邻查找。这里由于是离线模拟，所以我们导出所有待测试用户的表示向量，和所有物品的表示向量。

test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values, "movie_idx": item_profile['movie_id'].values}

# 以下两行是deepmatch中的通用使用方法，分别获得用户向量模型和物品向量模型
user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)
# 输入对应的数据拿到对应的向量
user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs)
print(user_embs.shape)
print(item_embs.shape)

In [ ]:
# 5. [可选的]如果有安装faiss库的同学，可以体验一下将上一步导出的物品向量构建索引，然后用用户向量来进行ANN查找并评估效果

test_true_label = {line[0]:[line[2]] for line in test_set}
import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N
index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(user_embs, 50)
s = []
hit = 0
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("recall", np.mean(s))
print("hr", hit / len(test_user_model_input['user_id']))